<a href="https://colab.research.google.com/github/AlessandroBigData/BD_Coursework/blob/master/Colab_9_Data_Validation_with_TensorFlow_Data_Validation_FINAL(TFDV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Install all dependencies and setting up the environment

In [2]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  python-snappy
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 10.8 kB of archives.
After this operation, 39.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-snappy amd64 0.5-1.1build2 [10.8 kB]
Fetched 10.8 kB in 0s (46.2 kB/s)
Selecting previously unselected package python-snappy.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../python-snappy_0.5-1.1build2_amd64.deb ...
Unpacking python-snappy (0.5-1.1build2) ...
Setting up python-snappy (0.5-1.1build2) ...


In [3]:
!pip install -q tensorflow_data_validation

     |████████████████████████████████| 2.4MB 2.9MB/s 
     |████████████████████████████████| 225kB 60.3MB/s 
     |████████████████████████████████| 2.8MB 40.5MB/s 
     |████████████████████████████████| 51kB 24.0MB/s 
     |████████████████████████████████| 51kB 22.9MB/s 
     |████████████████████████████████| 1.0MB 47.0MB/s 
     |████████████████████████████████| 153kB 63.6MB/s 
     |████████████████████████████████| 81kB 29.9MB/s 
     |████████████████████████████████| 143kB 58.7MB/s 
     |████████████████████████████████| 92kB 28.1MB/s 
     |████████████████████████████████| 102kB 34.3MB/s 
     |████████████████████████████████| 163kB 56.9MB/s 
     |████████████████████████████████| 92kB 31.7MB/s 
ERROR: google-cloud-storage 1.16.1 has requirement google-cloud-core<2.0dev,>=1.0.0, but you'll have google-cloud-core 0.29.1 which is incompatible.
ERROR: multiprocess 0.70.8 has requirement dill>=0.3.0, but you'll have dill 0.2.9 which is incompatible.
ERROR: google-cloud-tra

## Stage 2: Import project dependencies

In [0]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

from __future__ import print_function

## Stage 3: Simple dataset analysis

In [0]:
dataset = pd.read_csv("pollution-small.csv")

In [10]:
dataset.shape

(2188, 5)

In [0]:
training_data = dataset[:1600]

In [12]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [0]:
test_set = dataset[1600:]

In [14]:
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Stage 3: Data analysis and validation with TFDV

### Generate training data statistics

In [0]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset)

### Infering the schema

In [0]:
schema = tfdv.infer_schema(statistics=train_stats)

In [17]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'soot',FLOAT,required,,-
'no2',FLOAT,required,,-
'pm10',FLOAT,required,,-
'Date',BYTES,required,,-
'so2',FLOAT,required,,-


### Calculate test set statistics

In [32]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)
tfdv.visualize_statistics(test_stats)

## Stage 4: Compare test statistics with the Schema

### Checking for anomalies in new data

In [0]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

### Displaying all detected anomalies

- Integer larger than 10
- STRING type when expected INT type
- FLOAT type when expected INT type
- Integer smaller than 0

In [22]:
tfdv.display_anomalies(anomalies)

### New data WITH anomalies

In [0]:
test_set_copy = test_set.copy()

In [25]:
# Drop a Col and Shift the distribution
test_set_copy.drop("soot", axis=1, inplace=True)
test_set_copy.pm10 = test_set_copy.pm10 * 100000

test_set_copy.describe()

,pm10,no2,so2
count,5.880000e+02,588.000000,588.000000
mean,4.464825e+06,37.296922,13.605170
std,2.899209e+06,10.940050,5.098944
min,1.190000e+06,15.070000,4.990000
25%,2.833750e+06,29.217500,10.122500
50%,3.555500e+06,35.815000,12.345000
75%,5.081250e+06,43.872500,15.855000
max,2.737700e+07,106.030000,38.030000


In [34]:
# Shuffle cols
cols = test_set_copy.columns.tolist()
cols = cols[-1:] + cols[:-1]

print(cols)

['soot', 'Date', 'pm10', 'no2', 'so2']


In [0]:
# Add a new column
test_set_copy["new_col"] = 1

### Statistics based on data with anomalies

In [0]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)


In [0]:

anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [42]:
tfdv.display_anomalies(anomalies_new)

,Anomaly short description,Anomaly long description
Feature name,,
'new_col',New column,New column (column in data but not in schema)


## Stage 5: Prepare the schema for Serving

In [0]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

### Removing a target column from the Serving shema

In [0]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

### Checking for anomalies between the SERVING environment and new test set

In [0]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING")

In [0]:
tfdv.display_anomalies(serving_env_anomalies)